In [44]:
from googleapiclient.discovery import build
import pandas as pd
import csv

api_key = '------'
#set your API key
youtube = build('youtube', 'v3', developerKey=api_key)
# when call the function you need video ID
def scrape_comments_with_replies(youtube_id):
    data = []
    box = []
    #  Extract relevant information from the extract_comment_data 
    def extract_comment_data(comment_item, reply_count):
        name = comment_item["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
        comment = comment_item["snippet"]['topLevelComment']["snippet"]["textDisplay"]
        published_at = comment_item["snippet"]['topLevelComment']["snippet"]['publishedAt']
        likes = comment_item["snippet"]['topLevelComment']["snippet"]['likeCount']
        url = comment_item["snippet"]['topLevelComment']["snippet"]['authorProfileImageUrl']
        if 'authorChannelId' in comment_item["snippet"]['topLevelComment']["snippet"]:
            channel_id = comment_item["snippet"]['topLevelComment']["snippet"]['authorChannelId']['value']
        else:
            channel_id = ""
        id_user=comment_item['id']
        
        #Append the extracted data to the data list
        data.append({
            'Video ID': youtube_id,
            'Name': name,
            'Comment': comment,
            'Reply':"",
            'Date': published_at,
            'Likes': likes,
            'Reply Count': reply_count,
            'Profile Image Url': url,
            'User ID': channel_id,
            'Unique ID':id_user,
            'Parent ID': ""
        })
     # Extract relevant information from the extract_reply_data
    #in this function you need commenter ID
    def extract_reply_data(reply_item, parent_id):
        name = reply_item["snippet"]["authorDisplayName"]
        reply = reply_item["snippet"]["textDisplay"]
        published_at = reply_item["snippet"]['publishedAt']
        likes = reply_item["snippet"]['likeCount']
        url = reply_item["snippet"]['authorProfileImageUrl']
        channel_id = reply_item["snippet"]['authorChannelId']['value']
        id_user=reply_item["id"]
        parent_id=reply_item["snippet"]["parentId"]

        #append the extracted data to data list
        data.append({
            'Video ID': youtube_id,
            'Name': name,
            'Comment':"",
            'Reply': reply,
            'Date': published_at,
            'Likes': likes,
            'Reply Count': "",
            'Profile Image Url': url,
            'User ID': channel_id,
            'Unique ID':id_user,
            'Parent ID': parent_id
        })
        
    #Sends a request to the YouTube API to retrieve comment threads for a specific video ID (youtube_id) 
    response = youtube.commentThreads().list(
        part='snippet',
        videoId=youtube_id,
        maxResults='100',
        textFormat="plainText"
    ).execute()
    
    #The response contains a list of comment threads
    while "items" in response:
        # Executes a loop while there are still comment threads available in the response
        for item in response["items"]:
            #Iterates through each comment thread and extracts the comment and reply count
            comment = item["snippet"]['topLevelComment']["snippet"]["textDisplay"]
            reply_count = item["snippet"]['totalReplyCount']
            #Calls the extract_comment_data function to extract the relevant information from the comment
            extract_comment_data(item, reply_count)
            #if the reply count over 0, send the request to youtube comment resourse and usingthe parent ID
            if reply_count > 0:
                parent_id = item["snippet"]['topLevelComment']["id"]
                replies = youtube.comments().list(
                    part='snippet',
                    maxResults='100',
                    parentId=parent_id,
                    textFormat="plainText"
                ).execute()
            #Calls the extract_reply_data function to extract the relevant information from each reply
                for reply_item in replies["items"]:
                    extract_reply_data(reply_item, parent_id)
        #Checks if there is a next page token in the response, indicating there are more comment threads to fetch
        #If there is a next page token, sends a new request to the YouTube API to get the next page of comment threads
        #The loop continues until there are no more comment threads available
        if "nextPageToken" in response:
            next_page_token = response["nextPageToken"]
            response = youtube.commentThreads().list(
                part='snippet',
                videoId=youtube_id,
                pageToken=next_page_token,
                maxResults='100',
                textFormat="plainText"
            ).execute()
        else:
            break
    #Creates a pandas DataFrame from the collected data,
    #Saves the DataFrame as a CSV file named comments_{youtube_id}.csv
    #Returns the string "success" to indicate the successful completion of the function.
    df = pd.DataFrame(data)
    df.to_csv(f'comment/comments_{youtube_id}.csv')
    return "success" 

In [45]:
# ID.txt file consist of list of video IDs, create csv file each video in comment folder
video_ids = open('ID.txt', 'r')
file_list = list(video_ids)
for line in file_list:
    v_id = line.rstrip('\n')
    ID = "" + v_id + "";
    scrape_comments_with_replies(ID)
video_ids.close()